In [1]:
import os
import sys

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
sys.path.insert(0, project_root)

os.environ["MONGODB_URI"] = ""
os.environ["OPENAI_API_KEY"] = ""

In [2]:
from src.memorizz.memory_provider.mongodb.provider import MongoDBConfig, MongoDBProvider

# Create a memory provider
mongodb_config = MongoDBConfig(uri=os.environ["MONGODB_URI"])
memory_provider = MongoDBProvider(mongodb_config)

In [3]:
from src.memorizz.toolbox import Toolbox

# Create a toolbox
toolbox = Toolbox(memory_provider)

In [4]:
@toolbox.register_tool
def get_stock_price(ticker: str, currency: str = "USD") -> str:
    """
    Get the current stock price for a specified ticker symbol.

    Parameters:
    -----------
    ticker : str
        The stock ticker symbol (e.g., "AAPL" for Apple).
    currency : str, optional
        The currency in which the price should be returned (default is USD).

    Returns:
    --------
    str
        A description of the current stock price.
    """
    # Implementation here
    pass

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [5]:
# Get a tool by name
specific_tool = toolbox.get_tool_by_name("get_stock_price")
specific_tool


{'_id': ObjectId('67fc1260b0f9be100ffb6b2d'),
 'tool_id': '8cd2c240-0423-4feb-a380-55f0af496fa9',
 'type': 'function',
 'name': 'get_stock_price',
 'docstring': '\n    Get the current stock price for a specified ticker symbol.\n\n    Parameters:\n    -----------\n    ticker : str\n        The stock ticker symbol (e.g., "AAPL" for Apple).\n    currency : str, optional\n        The currency in which the price should be returned (default is USD).\n\n    Returns:\n    --------\n    str\n        A description of the current stock price.\n    ',
 'signature': "(ticker: str, currency: str = 'USD') -> str"}

In [6]:
# Get similar tools
similar_tools = toolbox.get_most_similar_tools("Get the stock price for Apple", limit=3)
similar_tools

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[{'_id': ObjectId('67fc1260b0f9be100ffb6b2d'),
  'tool_id': '8cd2c240-0423-4feb-a380-55f0af496fa9',
  'type': 'function',
  'name': 'get_stock_price',
  'docstring': '\n    Get the current stock price for a specified ticker symbol.\n\n    Parameters:\n    -----------\n    ticker : str\n        The stock ticker symbol (e.g., "AAPL" for Apple).\n    currency : str, optional\n        The currency in which the price should be returned (default is USD).\n\n    Returns:\n    --------\n    str\n        A description of the current stock price.\n    ',
  'signature': "(ticker: str, currency: str = 'USD') -> str",
  'score': 0.8110098838806152,
  'function': <function __main__.get_stock_price(ticker: str, currency: str = 'USD') -> str>}]

In [7]:
list_all_tools = toolbox.list_tools()
list_all_tools

[{'_id': ObjectId('67fc1260b0f9be100ffb6b2d'),
  'tool_id': '8cd2c240-0423-4feb-a380-55f0af496fa9',
  'type': 'function',
  'name': 'get_stock_price',
  'docstring': '\n    Get the current stock price for a specified ticker symbol.\n\n    Parameters:\n    -----------\n    ticker : str\n        The stock ticker symbol (e.g., "AAPL" for Apple).\n    currency : str, optional\n        The currency in which the price should be returned (default is USD).\n\n    Returns:\n    --------\n    str\n        A description of the current stock price.\n    ',
  'signature': "(ticker: str, currency: str = 'USD') -> str",
  'function': <function __main__.get_stock_price(ticker: str, currency: str = 'USD') -> str>}]

In [8]:
# Get a tool by id
tool_id = similar_tools[0]["tool_id"]
tool_retrieved_by_id = toolbox.get_tool_by_id(tool_id)
tool_retrieved_by_id

{'_id': ObjectId('67fc1260b0f9be100ffb6b2d'),
 'tool_id': '8cd2c240-0423-4feb-a380-55f0af496fa9',
 'type': 'function',
 'name': 'get_stock_price',
 'docstring': '\n    Get the current stock price for a specified ticker symbol.\n\n    Parameters:\n    -----------\n    ticker : str\n        The stock ticker symbol (e.g., "AAPL" for Apple).\n    currency : str, optional\n        The currency in which the price should be returned (default is USD).\n\n    Returns:\n    --------\n    str\n        A description of the current stock price.\n    ',
 'signature': "(ticker: str, currency: str = 'USD') -> str"}

In [9]:
updated_tool = toolbox.update_tool_by_id(tool_id, {"name": "get_stock_price_for_american_companies"})
updated_tool

True

In [10]:
retrieved_updated_tool = toolbox.get_tool_by_id(tool_id)
retrieved_updated_tool


{'_id': ObjectId('67fc1260b0f9be100ffb6b2d'),
 'tool_id': '8cd2c240-0423-4feb-a380-55f0af496fa9',
 'type': 'function',
 'name': 'get_stock_price_for_american_companies',
 'docstring': '\n    Get the current stock price for a specified ticker symbol.\n\n    Parameters:\n    -----------\n    ticker : str\n        The stock ticker symbol (e.g., "AAPL" for Apple).\n    currency : str, optional\n        The currency in which the price should be returned (default is USD).\n\n    Returns:\n    --------\n    str\n        A description of the current stock price.\n    ',
 'signature': "(ticker: str, currency: str = 'USD') -> str"}

In [11]:
# Delete a tool by name
success = toolbox.delete_tool_by_name("get_stock_price")
success

In [ ]:
# Delete a tool by id
success_deleting_tool_by_id = toolbox.delete_tool_by_id(tool_id)
success_deleting_tool_by_id


True

In [13]:
# Delete all tools
success_deleting_all_tools = toolbox.delete_all()
success_deleting_all_tools

False